<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D_3341_Find_Minimum_Time_to_Reach_Last_Room_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3341. Find Minimum Time to Reach Last Room I

https://leetcode.com/problems/find-minimum-time-to-reach-last-room-i/description/

There is a dungeon with `n x m` rooms arranged as a **grid**.

You are given a **2D array** `moveTime` of size `n x m`, where `moveTime[i][j]` represents the minimum time in seconds when you can start moving to that room.

You start from the room `(0, 0)` at time `t = 0` and *can move to an adjacent room*. Moving between adjacent rooms takes exactly **one second**.

Return the **minimum time** to reach the room `(n - 1, m - 1)`.

Two rooms are adjacent if they share a common wall, either horizontally or vertically.

**Constraints:**
- `2 <= n == moveTime.length <= 50`
- `2 <= m == moveTime[i].length <= 50`
- `0 <= moveTime[i][j] <= 10`<sup>`9`</sup>

**Example 1**

Explanation: The minimum time required is `6` seconds.
  
  - At time `t == 4`, move from room `(0, 0)` to room `(1, 0)` in one second.
  
  - At time `t == 5`, move from room `(1, 0)` to room `(1, 1)` in one second.

In [ ]:
test_case1 =  {
            "input": {
                "moveTime": [[0,4],[4,4]]
            },
            "output": 6
        }

**Example 2**

Explanation: The minimum time required is `3` seconds.

  - At time `t == 0`, move from room `(0, 0)` to room `(1, 0)` in one second.
  - At time `t == 1`, move from room `(1, 0)` to room `(1, 1)` in one second.
  - At time `t == 2`, move from room `(1, 1)` to room `(1, 2)` in one second.

In [ ]:
test_case2 =  {
            "input": {
                "moveTime": [[0,0,0],[0,0,0]]
            },
            "output": 3
        }

**Example 3**

In [ ]:
test_case3 =  {
            "input": {
                "moveTime": [[0,1],[1,2]]
            },
            "output": 3
        }

In [ ]:
test_cases = [test_case1, test_case2, test_case3]

In [ ]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Dijkstra's Shortest Path Algorithm - Beat 10.5%

### Observations

- Leetcode Hint: use shortest path algorithm to solve this problem

- Used algorithm: Dijkstra algorithm to find shortest path from source to all others vertices

**Mapping between graph problem and this problem**

- Let 2D-array `minTime` with size `n x m` is the array stores minimum time moving from source `(0,0)` to vertex `(i,j)`

- Each cell is a vertex. We have total `n x m` vertices. Adjacent cells of cell `a` are neighbor vertices of vertex `a`. A vertex and its neighbor have an edge between them.

- The time moving between cells is the edge's distance. In this case, it is `1`

- The minimum time can start moving to cell is the condition for moving to a vertex.

  - For example, moving from `a = (i,j)` to its neighbor vertex `b = (i+1,j)`.

  - We have `minTime[a] = x` and `moveTime[b] = y`

  - If `x + 1 < y` then we can not move from `a` to `b` with current minimum time of `a`

  - If the minimum time moving from all neighbor vertices of `b` to `b` are smaller than `moveTime[b]`, then the minimum time moving to `b` `minTime[b] = moveTime[b] + 1`. We start from `moveTime[b]` and take 1 sec to move to `b`

- Each time, we consider the un-processed vertex with minimum time moving from source to it. We using min heap data structure for this task.

### Analysis

- Time complexity:

  - Time to create edges: `O(n * m)`

  - Process all vertices:

    - For each vertex: we perform heappush `O(log(n * m))` and headpop `O(1)` (assume that the heap has maximum number of vertices `n * m`)

    - Process all vertices: `O(n * m * log(n * m))`

  - Total time: `O(n * m * log(n * m))`



### Implementation

In [ ]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOVING_TIME_BETWEEN_VERTICES = 1

    def minTimeToReach(self, moveTime: List[List[int]]) -> int:
        h = len(moveTime)
        w = len(moveTime[0])
        minTime = [[math.inf] * w for _ in range(h)]
        minTime[0][0] = 0
        edges = self.createEdges(h, w)

        minHeap = []

        # add source with minTime = 0
        heappush(minHeap, [0, (0,0)])

        processedVertices = set()

        # prettyPrintGrid(minTime)

        while minHeap:
            time, vertex = heappop(minHeap)

            if vertex in processedVertices:
                continue

            processedVertices.add(vertex)

            # print(f"processed_vertex: {vertex}")

            for neighborVertex in edges[vertex]:
                timeToNeighborVertex = max(self.MOVING_TIME_BETWEEN_VERTICES + time, moveTime[neighborVertex[0]][neighborVertex[1]]+1)

                minTime[neighborVertex[0]][neighborVertex[1]] = min(timeToNeighborVertex, minTime[neighborVertex[0]][neighborVertex[1]])

                heappush(minHeap, [timeToNeighborVertex, neighborVertex])

            # prettyPrintGrid(minTime)

        return minTime[h-1][w-1]


    def createEdges(self, h, w):
        edges = defaultdict(list)

        # inner grid, full four adjacent cells
        for i in range(1,h-1):
            for j in range(1,w-1):
                edges[(i,j)].append((i+1,j))
                edges[(i,j)].append((i-1,j))
                edges[(i,j)].append((i,j+1))
                edges[(i,j)].append((i,j-1))

        # four corner
        # top left (0,0)
        edges[(0,0)].append((0,1))
        edges[(0,0)].append((1,0))

        # top right (0,w-1)
        edges[(0,w-1)].append((0,w-2))
        edges[(0,w-1)].append((1,w-1))

        # bottom left (h-1, 0)
        edges[(h-1,0)].append((h-1,1))
        edges[(h-1,0)].append((h-2,0))

        # bottom right (h-1, w-1)
        edges[(h-1,w-1)].append((h-1,w-2))
        edges[(h-1,w-1)].append((h-2,w-1))

        # top edge
        # 0,j -> 0,j-1
        #     -> 0,j+1
        #     -> 1,j
        for j in range(1,w-1):
            edges[(0,j)].append((0,j-1))
            edges[(0,j)].append((0,j+1))
            edges[(0,j)].append((1,j))

        # bottom edge
        for j in range(1,w-1):
            edges[(h-1,j)].append((h-1,j-1))
            edges[(h-1,j)].append((h-1,j+1))
            edges[(h-1,j)].append((h-2,j))

        # left edge
        for i in range(1,h-1):
            edges[(i,0)].append((i+1,0))
            edges[(i,0)].append((i-1,0))
            edges[(i,0)].append((i,1))

        # right edge
        for i in range(1,h-1):
            edges[(i,w-1)].append((i+1,w-1))
            edges[(i,w-1)].append((i-1,w-1))
            edges[(i,w-1)].append((i,w-2))

        return edges

def prettyPrintGrid(grid):
    for row in grid:
        print('\t'.join([str(_) for _ in row]))

In [ ]:
run_test_cases(Solution(), "minTimeToReach")

In [ ]:
run_test_case(Solution(), "minTimeToReach", 2)

# Approach 2 - Optimized approach 1 - Beat 60%

### Observations

- Instead of pre-compute edges, we can compute vertex's neighbors on the fly when process vertex. We use a delta array and apply it to current vertex to find its four neightbor vertices

- We do not to store processed vertices. We can determine if it is processed or not by checking its current `minTime` with initialized value.

- Return early if we encounter `(n-1, m-1)` vertex

### Analysis

- Time complexity:

  - Time to create edges: `O(n * m)`

  - Process all vertices:

    - For each vertex: we perform heappush `O(log(n * m))` and headpop `O(1)` (assume that the heap has maximum number of vertices `n * m`)

    - Process all vertices: `O(n * m * log(n * m))`

  - Total time: `O(n * m * log(n * m))`



In [49]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOVING_TIME_BETWEEN_VERTICES = 1

    def minTimeToReach(self, moveTime: List[List[int]]) -> int:
        n = len(moveTime)
        m = len(moveTime[0])
        minTime = [[math.inf] * m for _ in range(n)]
        minTime[0][0] = 0

        minHeap = []

        # add source (minTime, i, j)
        heappush(minHeap, (0, 0, 0))

        # prettyPrintGrid(minTime)
        neighborDirections = [(0,1), (0,-1), (1,0), (-1,0)]

        while minHeap:
            time, i, j = heappop(minHeap)

            # early return
            if i == n - 1 and j == m - 1:
                return time

            for delta_i, delta_j in neighborDirections:
                neighbor_i = i + delta_i
                neighbor_j = j + delta_j

                if 0 <= neighbor_i < n and 0 <= neighbor_j < m:
                    timeToNeighborVertex = max(time, moveTime[neighbor_i][neighbor_j]) + self.MOVING_TIME_BETWEEN_VERTICES

                    updatedMinTime = min(timeToNeighborVertex, minTime[neighbor_i][neighbor_j])

                    if minTime[neighbor_i][neighbor_j] == math.inf:
                        heappush(minHeap, [timeToNeighborVertex, neighbor_i, neighbor_j])

                    minTime[neighbor_i][neighbor_j] = updatedMinTime

In [50]:
run_test_cases(Solution(), "minTimeToReach")